# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [1]:
import re
import json
from os.path import dirname, join, abspath

import pandas

from ecolime.flat_files import ecoli_files_dir
import cobra.test

from minime import *
from ecolime.flat_files import *
from ecolime import flat_files
from minime.util.building import build_reactions_from_genbank, add_transcription_reaction
import ecolime

## Begin by loading metabolites

In [2]:
def fix_id(id_str):
    return id_str.replace("_DASH_", "__")

In [3]:
def try_compartment(id_str):
    id_str = id_str.replace("_DASH_", "__")
    try:
        me.metabolites.get_by_id(id_str + '_c')
        return id_str + '_c'
    except:
        return id_str

In [4]:
me = MEmodel("iJO1366-ME")
me.id = "iJO1366-ME"
me.compartments = {"p": "Periplasm", "e": "Extra-organism",
                   "c": "Cytosol"}
compartment_lookup = {v: k for k, v in me.compartments.items()}

In [5]:
met_info = pandas.read_csv(
    join(ecoli_files_dir, "metabolites.txt"),
    delimiter="\t", header=None, index_col=0,
    names=["id", "name", "formula", "compartment", "data_source"])

In [6]:
for met_id in met_info.index:
    fixed_id = fix_id(met_id)
    for compartment in met_info.compartment[met_id].split("AND"):
        compartment = compartment.strip()
        if met_id == "dpm":
            print "Assigned %s to c" % met_id
            compartment = me.compartments["c"]
        elif compartment == "No_Compartment":
            print "Assigned %s to e" % met_id
            compartment = me.compartments["e"]
        metab = Metabolite(fixed_id + "_" + compartment_lookup[compartment])
        metab.name = met_info.name[met_id]
    
        metab.formula = met_info.formula[met_id]
        me.add_metabolites(metab)

Assigned tl to e
Assigned hemed to e
Assigned pqq to e
Assigned dpm to c
Assigned 23bpg to e
Assigned tqn to e


Add generic ions to simplify complexes

## Build Metabolic reactions

In [7]:
reaction_info = get_reaction_info_frame()
reaction_dict = get_reaction_matrix_dict()

In [8]:
for r_id in reaction_info.index:
    reaction = StoichiometricData(r_id, me)
    reaction._stoichiometry = {try_compartment(k): v\
                               for k, v in reaction_dict[r_id].items()}
    reaction.lower_bound = \
        -1000. if reaction_info.is_reversible[r_id] else 0.
    reaction.upper_bound = 1000.

Also make a dummy reaction

In [9]:
dummy = StoichiometricData("dummy_reaction", me)
dummy.lower_bound = 0
dummy.upper_bound = 1000
dummy._stoichiometry = {}

Boundary Reactions

In [10]:
sources_sinks = pandas.read_csv(join(ecoli_files_dir, "reaction_matrix_sources_and_sinks.txt"), delimiter="\t",
                                header=None, names=["rxn_id", "met_id", "compartment", "stoic"], index_col=1)
sources_sinks.index = [fix_id(i) for i in sources_sinks.index]

source_amounts = pandas.read_csv(join(ecoli_files_dir, "exchange_bounds.txt"),
                                 delimiter="\t", index_col=0, names=["met_id", "amount"])
source_amounts.index = [fix_id(i) for i in source_amounts.index]


for met_id in sources_sinks.index:
    model_met_id = met_id + "_" + compartment_lookup[sources_sinks.compartment[met_id]]
    # EX_ or DM_ + met_id
    reaction_id = sources_sinks.rxn_id[met_id][:3] + model_met_id
    reaction = cobra.Reaction(reaction_id)
    me.add_reaction(reaction)
    reaction.add_metabolites({me.metabolites.get_by_id(model_met_id): -1})
    # set bounds on exchanges
    if reaction.id.startswith("EX_") and met_id in source_amounts.index:
        reaction.lower_bound = -source_amounts.amount[met_id]

Set bounds on boundary reactions

## Build ribosome and RNA Polymerase

In [11]:
ribosome_complex = ComplexData("ribosome", me)
ribosome_components = ribosome_complex.stoichiometry
for i in ["b3851", "b3854", "b3855"]:
    ribosome_components["RNA_" + i] = 1
# 30S Listed as [rpsA -rpsU], sra 
# [rplA-rplF],  rplI, [rplK-rplY],
# [rpmA-rpmJ]

for i in ["b0911", "b0169", "b3314", "b3296", "b3303", "b4200", "b3341",
          "b3306", "b3230", "b3321", "b3297", "b3342", "b3298", "b3307", 
          "b3165", "b2609", "b3311", "b4202", "b3316", "b0023", "b3065",
          "b1480"]:
    ribosome_components["protein_" + i] = 1
    
# 50S listed as [rplA-rplF],rplJ, rplI, rplK [rplM-rplY], [rpmA-rpmJ]    
for i in ["b3984", "b3317", "b3320", "b3319", "b3308", "b3305", 
          "b3958", "b4203", "b3983", "b3231", "b3310", "b3301",
          "b3313", "b3294", "b3304", "b2606", "b1716", "b3186",
          "b3315", "b3318", "b3309", "b2185", 
          "b3185", "b3637", "b3312", "b3302", "b3936", "b1089", 
          "b3636", "b3703", "b1717", "b3299"]:
    ribosome_components["protein_" + i] = 1
# [rplJ, 2(2[rplL7])]
ribosome_components["protein_" + "b3986"] = 4

ribosome_complex.create_complex_formation()

Created <TranslatedGene protein_b1089 at 0x7fa8cd6e0550> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b3637 at 0x7fa8cd6e0590> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b3636 at 0x7fa8cd6e05d0> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b3304 at 0x7fa8cd6e0410> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b4200 at 0x7fa8cd6e0450> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b1716 at 0x7fa8cd6e0150> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b1717 at 0x7fa8cd6e0190> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b0023 at 0x7fa8cd6e01d0> in <ComplexFormation formation_ribosome at 0x7fa8cd6e04d0>
Created <TranslatedGene protein_b3306 at 0x7fa8cd6e0210> in <ComplexForm

In [12]:
RNAP_complex = ComplexData("RNA_Polymerase", me)
RNAP_components = RNAP_complex.stoichiometry
# Core RNA Polymerase Enzyme
for i in {"b3295" : "rpoA", "b3988" : "rpoC", "b3987" : "rpoB"}:
    if i == "b3295":
        RNAP_components["protein_" + i] = 2
    else:
        RNAP_components["protein_" + i] = 1

RNAP_complex.create_complex_formation()

Created <TranslatedGene protein_b3295 at 0x7fa8cd6d1b90> in <ComplexFormation formation_RNA_Polymerase at 0x7fa8cd6d1bd0>
Created <TranslatedGene protein_b3988 at 0x7fa8cd6d1cd0> in <ComplexFormation formation_RNA_Polymerase at 0x7fa8cd6d1bd0>
Created <TranslatedGene protein_b3987 at 0x7fa8cd6d1d10> in <ComplexFormation formation_RNA_Polymerase at 0x7fa8cd6d1bd0>


### Add Transcription/Translation from the genbank file

In [13]:
build_reactions_from_genbank(me, join(ecoli_files_dir, "NC_000913.2.gb"))

TODO deal with selenocystine


Add a dummy protein in as well

In [14]:
dna_sequence = "ATG" + "TTT" * 5 + "TAT"*5+ "ACG"*5 + "GAT" *5 + "AGT"*5 + "TAA"
add_transcription_reaction(me, "dummy", {"dummy"}, dna_sequence)
me.add_metabolites(TranslatedGene("protein_" + "dummy"))
translation = TranslationReaction("translation_" + "dummy")
me.add_reaction(translation)
translation.translation_data = TranslationData("dummy", me, "RNA_dummy", "protein_dummy")
translation.translation_data.compute_sequence_from_DNA(dna_sequence)
translation.update()

complex_data = ComplexData("CPLX_dummy", me)
complex_data.stoichiometry = {}
complex_data.stoichiometry["protein_" + "dummy"] = 1
complex_data.create_complex_formation()

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [15]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [16]:
# ME_complex_dict is a dict of {'complex_id': ['bnum(count)']}
ME_complex_dict = get_complex_to_bnum_dict()
# some entries in the file need to be renamed
renames = {"MnmE_": "b3706", "MnmG_": "b3741", "YheM_": "b3344", "YheL_": "b3343", "YheN_": "b3345"}
rna_components = {"b3123"}  # component id should have RNA_ instead of protein_

for cplx, value in ME_complex_dict.iteritems():
    complex_data = ComplexData(cplx, me)
    for gene in value:
        stoichiometry = gene[6]
        bnum = gene[0:5]
        comp_id = "RNA_" + bnum if bnum in rna_components \
            else "protein_" + renames.get(bnum, bnum)
        try:
            complex_data.stoichiometry[comp_id] = float(stoichiometry)
        except:
            complex_data.stoichiometry[comp_id] = float(1)

In [17]:
# {modified_complex_id: ['unmodified_complex_id', {component_id: stoic}]
modification_dict = get_protein_modification_dict(generic=False)
for mod_cplx_id, mod_complex_info in iteritems(modification_dict):
    unmod_cplx_id, mods = mod_complex_info
    unmod_cplx = me.complex_data.get_by_id(unmod_cplx_id)
    cplx = ComplexData(mod_cplx_id, me)
    cplx.stoichiometry = unmod_cplx.stoichiometry
    cplx.translocation = unmod_cplx.translocation
    cplx.chaperones = unmod_cplx.chaperones
    cplx.unmod_cplx_id = unmod_cplx_id
    if len(set(mod_cplx_id.split("_mod_")[1:])) == len(mods):
        for mod_comp, mod_count in iteritems(mods):
            mod_id = "mod_" + mod_comp
            try:
                mod = me.modification_data.get_by_id(mod_id)
            except:
                mod = ModificationData(mod_id, me)
                mod.stoichiometry = {mod_comp: -1}
            cplx.modifications[mod_id] = -mod_count
    else:
        print "TODO:", mod_cplx_id

In [18]:
me.complex_data.get_by_id('ACETYLORNTRANSAM-CPLX').stoichiometry

{'protein_b3359': 2.0}

In [19]:
me.complex_data.get_by_id('ACETYLORNTRANSAM-CPLX_mod_pydx5p').stoichiometry

{'protein_b3359': 2.0}

Some modifications are enzyme-catalyzed

TODO: Fe-S enzyme-catalyzed modifications

In [20]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_oc':-1,
                          'amet_c':-2,
                          'lipoamp_c':-1,
                          'met__L_c':2,
                          'dad__5_c':2,
                          'CPLX0-782_mod_2:4fe4s':-1,
                          'CPLX0-782_mod_1:2fe2s_mod_1:4fe4s':1,
                          'fe2_c':2,
                          'h_c':-2,
                          'nad_c':-1,
                          'nadh_c':1,
                          'EG50003-MONOMER_mod_pan4p':1}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [21]:
# todo bmocogdp_c mods
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER',
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER',
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', 
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER',
                 'NAP-CPLX':'NAPD-MONOMER',
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'}

Crutch reactions for mets that are blocked. TODO remove

In [22]:
target_list = ['4fe4s_c', 'LI_c', '2fe2s_c', '3fe4s_c', 'pqq_c',
               'hemed_c', 'acetyl_c', 'NiFeCoCN2_c', 'SH_c',
               '23bpg_c', 'dpm_c', 'tl_c']
target_list.extend(
    ['EG50003-MONOMER_mod_pan4p', 'CPLX0-782_mod_2:4fe4s',
     'EG50003-MONOMER_mod_pan4p_mod_oc', 'CPLX0-782_mod_1:2fe2s_mod_1:4fe4s'])

for met_id in target_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite '4fe4s_c' created
unknown metabolite 'LI_c' created
unknown metabolite '2fe2s_c' created
unknown metabolite '3fe4s_c' created
unknown metabolite 'pqq_c' created
unknown metabolite 'hemed_c' created
unknown metabolite 'acetyl_c' created
unknown metabolite 'NiFeCoCN2_c' created
unknown metabolite 'SH_c' created
unknown metabolite '23bpg_c' created
unknown metabolite 'tl_c' created
unknown metabolite 'EG50003-MONOMER_mod_pan4p' created
unknown metabolite 'CPLX0-782_mod_2:4fe4s' created
unknown metabolite 'EG50003-MONOMER_mod_pan4p_mod_oc' created
unknown metabolite 'CPLX0-782_mod_1:2fe2s_mod_1:4fe4s' created


Build all complex formation reactions

TODO: This shouldn't be prining out stuff. Fix them.

In [23]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

modification_EG10370-MONOMER_mod_mn2
modification_AAS-MONOMER_mod_mg2
modification_AROL-MONOMER_mod_mn2
modification_CPLX0-721_mod_mn2
modification_PSERPHOSPHA-MONOMER_mod_mg2
modification_ATPSYN-CPLX_mod_mg2
modification_PD04413_mod_cd2
modification_GLYCEROL-KIN-CPLX_mod_mg2
modification_Rnd_mono_mod_5:mg2
modification_G7502-MONOMER_mod_zn2
modification_GMP-SYN-CPLX_mod_mg2
modification_AROL-MONOMER_mod_mg2
modification_GLYCOPHOSPHORYL-CPLX_mod_pydx5p
modification_PEPSYNTH-CPLX_mod_mg2
modification_hRNAP_mod_1:zn2_mod_2:mg2
modification_LysII_RS_dim_mod_lys_DASH_L
modification_Trp_RS_dim_mod_mg2
modification_CPLX0-7615_mod_mg2
modification_FUMARASE-B_mod_4fe4s
modification_GTP-CYCLOHYDRO-II-CPLX_mod_mg2
modification_Ala_RS_tetra_mod_4:zn2
modification_UDP-NACMURALA-GLU-LIG-MONOMER_mod_k_mod_mg2
modification_ADENYLOSUCCINATE-SYN-DIMER_mod_mg2
modification_AROK-MONOMER_mod_k
modification_CPLX0-250_FORMATEDEHYDROGH-MONOMER_mod_8:4fe4s_mod_2:2fe2s_mod_bmocogdp_mod_NiFeCoCN2
modification_T

In [24]:
from collections import defaultdict

In [25]:
a = defaultdict(int)

In [26]:
isinstance(a,defaultdict)

True

## Associate Complexes with Reactions

In [27]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = flat_files.get_reaction_to_modified_complex(generic=False)

Fixed _DASH:  D__LACtex
Fixed _DASH:  L__LACD2
Fixed _DASH:  L__LACD3
Fixed _DASH:  D__LACt2pp
Fixed _DASH:  L__LACtex
Fixed _DASH:  L__LACt2rpp


In [28]:
for reaction_data in me.stoichiometric_data:
    # Some reactions have no complex associated. Determine if they are
    # spontaneous or orphan reactions.
    try:
        complexes = rxnToModCplxDict[reaction_data.id]
    except KeyError:
        # These are orphans catalyzed by a dummy
        if reaction_data.id == "dummy_reaction" or \
                not reaction_info.is_spontaneous[reaction_data.id]:
            complexes = ["CPLX_dummy"]
        # These are truly spontaneous
        else:
            complexes = [None]
    for complex_id in complexes:
        complex_data = me.complex_data.get_by_id(complex_id) if complex_id else None
        if reaction_data.lower_bound < 0:
            r = MetabolicReaction(reaction_data.id + "_REV_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = True
            r.update(create_new=True)
        if reaction_data.upper_bound > 0:
            r = MetabolicReaction(reaction_data.id + "_FWD_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = False
            r.update(create_new=True)

Created <Metabolite CPLX0-7817_mod_Oxidized at 0x7fa8caa84590> in <MetabolicReaction RNDR1b3_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa8caa84550>
Created <Metabolite GLUTAREDOXIN-MONOMER_mod_Oxidized at 0x7fa8caa84610> in <MetabolicReaction RNDR1b1_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa8caa845d0>
Created <Metabolite GRXC-MONOMER_mod_Oxidized at 0x7fa8caa84690> in <MetabolicReaction RNDR1b4_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa8caa84650>
Created <Metabolite GRXB-MONOMER_mod_Oxidized at 0x7fa8caa84810> in <MetabolicReaction RNDR1b2_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa8caa847d0>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_3ha at 0x7fa8caa84e10> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7fa8caa84dd0>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_act at 0x7fa8caa84e50> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7fa8caa84dd0>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_ddca at 0x7fa8caa84fd0> in <Me

In [29]:
# This reaction is weird
me.reactions.get_by_id('CITLY-CPLX_2tpr3dpcoa_FWD_G6340-MONOMER').reaction

'4.27350427350427e-6*mu G6340-MONOMER + 2tpr3dpcoa_c --> '

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [30]:
for r in me.reactions:
    if isinstance(r, tRNAChargingReaction):
        r.update()
for r in me.reactions:
    if isinstance(r, TranslationReaction):
        r.update()
    if isinstance(r, TranscriptionReaction):
        r.update()
    if isinstance(r, MetabolicReaction):
        r.update()

Remove unused protein and mRNA to make the model solve faster (TODO remove unused complexes too)

In [31]:
for p in me.metabolites.query("protein"):
    if len(p._reaction) == 1:
        list(p._reaction)[0].delete(remove_orphans=True)
for m in me.metabolites.query("RNA"):
    if len(m._reaction) == 1:
        list(m._reaction)[0].delete(remove_orphans=True)

This gives the total number of genes included

In [32]:
len(me.reactions.query("transcription"))

1648

In [33]:
for met in me.metabolites:
    if len(met.reactions) == 1:
        print met

cpg180_c
selnp_c
ragund_c
23doguln_c
4mhetz_c
4hthr_c
cpe160_c
gdpfuc_c
cinnm_c
preq1_c
aconm_c
didp_c
hqn_c
malt6p_c
thmnp_c
n8aspmd_c
dxyl_c
mercppyr_c
mi1p__D_c
mococdp_c
met__D_c
2pglyc_c
acanth_c
xdp_c
na15dap_c
cpg160_c
sarcs_c
cpe180_c
cs_e
athtp_c
ap5a_c
aso4_c
dmbzid_c
Nmtrp_c
gp4g_c
ag_c
N1aspmd_c
tungs_c
Rrf_mono
IscS
YhhP_mono
RpoS_mono
RrmJ_mono
RplC_mono
RpsP_mono
YrdC_mono
NARW-MONOMER
NusG_mono
#PrmB_mono
CITC-MONOMER
RpmA_mono
RpsR_mono
RpsF_mono
InfA_mono
ThiI_mono
#RimJ_mono
YecO_mono
TruA_dim
RpsJ_mono
RsmG_mono
TmcA_mono
YjbC_mono
EG11597-MONOMER
#CPLX0-7805
#YcaO_mono
RpmI_mono
RsmI_mono
RplE_mono
G6712-MONOMER
RplQ_mono
#Dtd_dim
NusA_mono
#YbbB_dim
RpoH_mono
RpsN_mono
RplM_mono
Sra_mono
InfC_mono
TufB_mono
TruB_mono
RNase_G_dim
RplU_mono
YggJ_dim
DusC_mono
MnmG_dim
#RimI_mono
EG12450-MONOMER
YheL_mono
YggH_mono
RlmI_dim
YheM_mono
RpmJ_mono
RpsM_mono
RpsQ_mono
RlmG_mono
RpsS_mono
RpmC_mono
#RimL_mono
RlmM_mono
RplR_mono
RpsO_mono
RpsB_mono
RlmB_dim
RsmD_mono
EG119

## Attempt to set keffs

In [34]:
divalent_list = flat_files.divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    for i in divalent_list: 
        keff = keff.replace(i,'generic_divalent')
    for i in monovalent_list: 
        keff = keff.replace(i, '_mod_generic_monovalent')
    keffs[keff] = value
    
for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        print r.id, "has no associated complex"
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

modification_EG10370-MONOMER_mod_mn2 has no associated complex
modification_AAS-MONOMER_mod_mg2 has no associated complex
modification_AROL-MONOMER_mod_mn2 has no associated complex
modification_CPLX0-721_mod_mn2 has no associated complex
modification_PSERPHOSPHA-MONOMER_mod_mg2 has no associated complex
modification_ATPSYN-CPLX_mod_mg2 has no associated complex
modification_PD04413_mod_cd2 has no associated complex
modification_GLYCEROL-KIN-CPLX_mod_mg2 has no associated complex
modification_Rnd_mono_mod_5:mg2 has no associated complex
modification_G7502-MONOMER_mod_zn2 has no associated complex
modification_GMP-SYN-CPLX_mod_mg2 has no associated complex
modification_AROL-MONOMER_mod_mg2 has no associated complex
modification_GLYCOPHOSPHORYL-CPLX_mod_pydx5p has no associated complex
modification_PEPSYNTH-CPLX_mod_mg2 has no associated complex
modification_LysII_RS_dim_mod_lys_DASH_L has no associated complex
modification_Trp_RS_dim_mod_mg2 has no associated complex
modification_CPLX0-

## Solve

In [35]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.
#me.reactions.get_by_id('PYDXK_FWD_PDXK-CPLX').objective_coefficient = 1

In [36]:
from minime.solve.algorithms import binary_search, fva
from minime.solve.symbolic import compile_expressions

In [37]:
exch_list = []
for rxn in me.reactions:
    if rxn.startswith('EX_'):
        rxn.upper_bound = 0.
        #rxn.objective_coefficient = 1.
        exch_list.append(rxn.id)
for rxn in exch_list:
    met = me.reactions.get_by_id(rxn).metabolites.keys()[0]
    r = Reaction('EX_' + met.id + '_f')
    r.add_metabolites({met:-1})
    r.objective_coefficient = 0.
    r.lower_bound = 0.
    r.upper_bound = 1000.
    me.add_reaction(r)

In [38]:
expressions = compile_expressions(me)

In [41]:
binary_search(me, min_mu=0, max_mu=.001,mu_accuracy=1e-6, compiled_expressions=expressions)

0.0000000000000	1000.0
0.0010000000000
0.0005000000000
0.0002500000000
0.0001250000000
0.0000625000000


KeyboardInterrupt: 

In [ ]:
def find_met_reactions(met_id):
    return [i for i in me.metabolites.get_by_id(met_id).reactions]
            #if not isinstance(i, (TranslationReaction, TranscriptionReaction))]

In [ ]:
fva_results = fva(me, .5, find_met_reactions("ca2_c"), compiled_expressions=expressions)
res = pandas.DataFrame.from_dict(fva_results).T
res

In [ ]:
me.reactions.ca2_c_to_generic.reaction

In [ ]:
me.reactions.get_by_id('DX_PORPHOBILSYNTH-CPLX_mod_zn').reaction

In [ ]:
len(me.reactions)

In [ ]:
fva_results = fva(me, .5, find_met_reactions("zn2_c"), compiled_expressions=expressions)
res = pandas.DataFrame.from_dict(fva_results).T
res

In [ ]:
me.reactions.get_by_id('translation_b0369').reaction

In [ ]:
me.reactions.MOX_REV_CPLX_dummy.knock_out()
me.reactions.MOX_FWD_CPLX_dummy.knock_out()

In [ ]:
sol = solve_at_growth_rate(me, .5, compiled_expressions=expressions, objective_sense='maximize')
sol.f

In [ ]:
for key, value in sol.x_dict.items():
    if key.startswith('EX_') and value < -.01:
        print key, value

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux()
view.display_in_notebook()

In [ ]:
import json
f = open('/home/sbrg-cjlloyd/Desktop/failure.json','w')
json.dump(a,f)
f.close()

In [ ]:
me.reactions.get_by_id('GLCDpp_FWD_G6437-MONOMER_mod_generic_divalent_mod_pqq').reaction

In [ ]:
me.reactions.get_by_id('formation_G6437-MONOMER_mod_generic_divalent_mod_pqq').reaction

In [ ]:
me.reactions.EX_so4_e.lower_bound

In [ ]:
me.reactions.PDH_FWD_PYRUVATEDEH-CPLX_mod_generic_divalent_mod_fad_mod_thmpp_mod_lipo

In [ ]:
me.reactions.get_by_id('formation_PYRUVATEDEH-CPLX_mod_generic_divalent_mod_fad_mod_thmpp_mod_lipoalt').reaction

In [ ]:
me.metabolites.lipoamp_c.reactions

In [ ]:
import pandas

In [ ]:
old_sim = pandas.read_pickle("/home/aebrahim/old_me/me/pandas_frames/all_metabolic_fluxes_v2")

In [ ]:
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = old_sim.base_plus_Adenine_keff_v2.to_dict()
view.display_in_notebook()

In [ ]:
me.compute_solution_error()

In [ ]:
#me.reactions.S_transfer_to_SufBCD_mod_2:fe2_FWD_CPLX_dummy
for reaction in me.reactions:
    if reaction.endswith('_'):
        print reaction.id

In [ ]:
cobra.